In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from matplotlib import pyplot as plt
import json
from shapely.geometry import Point


In [2]:
with open('basman_cafes.json', 'r', encoding = 'utf-8') as f:
    cafes = json.load(f)
with open('basman_complete_education.json', 'r', encoding = 'utf-8') as f:
    education = json.load(f)

with open('basman_leisure_data.json', 'r', encoding = 'utf-8') as f:
    leisure = json.load(f)

with open('basman_medical.json', 'r', encoding = 'utf-8') as f:
    medical = json.load(f)

with open('basman_residential_data.json', 'r', encoding = 'utf-8') as f:
    residential = json.load(f)

with open('basman_retail_data.json', 'r', encoding = 'utf-8') as f:
    retail = json.load(f)

with open('basman_transport_data.json', 'r', encoding = 'utf-8') as f:
    transport = json.load(f)

with open('basmannyy_transport_and_offices.json', 'r', encoding = 'utf-8') as f:
    transp_offices = json.load(f)

cafes = pd.DataFrame(cafes)
education = pd.DataFrame(education)
leisure = pd.DataFrame(leisure)
medical = pd.DataFrame(medical)
residential = pd.DataFrame(residential)
retail = pd.DataFrame(retail)
transport = pd.DataFrame(transport)


all_dfs = [cafes, education, leisure, medical, residential, retail, transport]
for df in all_dfs:

    lon  = df['coordinates'].apply(lambda x: x.get('longitude'))
    lat = df['coordinates'].apply(lambda x: x.get('latitude'))
    df['lon'] = lon
    df['lat'] = lat
    df = df.drop('coordinates', axis = 1)

all_objects = []

for key, value in transp_offices.items():
    if len(value) == 0:
        continue
    df = pd.json_normalize(value)
    df["category"] = key
    all_objects.append(df)

combined_df = pd.concat(all_objects, ignore_index=True)
combined_df = combined_df[['id', 'type', 'name', 'lat', 'lon', 'category']]

combined_gdf = gpd.GeoDataFrame(
    combined_df,
    geometry=gpd.points_from_xy(combined_df['lon'], combined_df['lat']),
    crs='EPSG:4326'
)
combined_df.head()

,id,type,name,lat,lon,category
0,242546356,node,Бауманская,55.773039,37.680549,metro_stations
1,1123660474,node,Курская,55.757198,37.659332,metro_stations
2,5176322725,node,Китай-город,55.755652,37.633565,metro_stations
3,5202107570,node,Курская,55.758019,37.658419,metro_stations
4,5202107572,node,Чкаловская,55.756702,37.656998,metro_stations


In [3]:
combined_df.category.unique()

array(['metro_stations', 'bus_stops', 'tram_stops', 'office_places'],
      dtype=object)

In [4]:
combined_df

,id,type,name,lat,lon,category
0,242546356,node,Бауманская,55.773039,37.680549,metro_stations
1,1123660474,node,Курская,55.757198,37.659332,metro_stations
2,5176322725,node,Китай-город,55.755652,37.633565,metro_stations
3,5202107570,node,Курская,55.758019,37.658419,metro_stations
4,5202107572,node,Чкаловская,55.756702,37.656998,metro_stations
...,...,...,...,...,...,...
1206,17594392,relation,None,55.778617,37.696393,office_places
1207,17864576,relation,None,55.753670,37.666680,office_places
1208,19558829,relation,None,55.756574,37.633859,office_places
1209,19558831,relation,Российский федеральный центр судебной экспертизы,55.756905,37.634060,office_places


In [5]:
to_m = lambda gdf: gdf.to_crs(epsg=3857)

cafes_gdf = gpd.GeoDataFrame(
    cafes,
    geometry=gpd.points_from_xy(cafes['lon'], cafes['lat']),
    crs='EPSG:4326'
)
education_gdf = gpd.GeoDataFrame(
    education,
    geometry=gpd.points_from_xy(education['lon'], education['lat']),
    crs='EPSG:4326'
)

leisure_gdf = gpd.GeoDataFrame(
    leisure,
    geometry=gpd.points_from_xy(leisure['lon'], leisure['lat']),
    crs='EPSG:4326'
)

medical_gdf = gpd.GeoDataFrame(
    medical,
    geometry=gpd.points_from_xy(medical['lon'], medical['lat']),
    crs='EPSG:4326'
)
residential_gdf = gpd.GeoDataFrame(
    residential,
    geometry=gpd.points_from_xy(residential['lon'], residential['lat']),
    crs='EPSG:4326'
)
retail_gdf = gpd.GeoDataFrame(
    retail,
    geometry=gpd.points_from_xy(retail['lon'], retail['lat']),
    crs='EPSG:4326'
)

retail_gdf = to_m(retail_gdf)
residential_gdf = to_m(residential_gdf)
medical_gdf = to_m(medical_gdf)
leisure_gdf = to_m(leisure_gdf)
education_gdf = to_m(education_gdf)
cafes_gdf = to_m(cafes_gdf)
combined_gdf = to_m(combined_gdf)

metro_gdf = combined_gdf[combined_gdf['category'] == 'metro_stations']
bus_gdf = combined_gdf[combined_gdf['category'] == 'bus_stops']
tram_gdf = combined_gdf[combined_gdf['category'] == 'tram_stops']
office_gdf = combined_gdf[combined_gdf['category'] == 'office_places']

def min_distance(src, target):
    """ минимальная дистанция до объекта-категории"""
    return src.geometry.apply(lambda p: target.distance(p).min() if len(target) > 0 else None)


def count_within_radius(src_gdf, target_gdf, radius_m=500):
    """Считает, сколько объектов target_gdf попадает в буфер радиусом radius_m от каждой точки src_gdf.
    Возвращает Series с количеством для каждой строки src_gdf.
    """
    buffer_gdf = src_gdf.copy()
    buffer_gdf['geometry'] = buffer_gdf.buffer(radius_m)

    joined = gpd.sjoin(target_gdf, buffer_gdf, predicate='within')

    counts = joined.groupby('index_right').size()
    return src_gdf.index.map(counts).fillna(0)

cafes_gdf['dist_to_retail'] = min_distance(cafes_gdf, retail_gdf)
cafes_gdf['dist_to_residential'] = min_distance(cafes_gdf, residential_gdf)
cafes_gdf['dist_to_medical'] = min_distance(cafes_gdf, medical_gdf)
cafes_gdf['dist_to_leisure'] = min_distance(cafes_gdf, leisure_gdf)
cafes_gdf['dist_to_education'] = min_distance(cafes_gdf, education_gdf)
cafes_gdf['dist_to_metro'] = min_distance(cafes_gdf, metro_gdf)
cafes_gdf['dist_to_bus'] = min_distance(cafes_gdf, bus_gdf)
cafes_gdf['dist_to_tram'] = min_distance(cafes_gdf, tram_gdf)
cafes_gdf['dist_to_office'] = min_distance(cafes_gdf, office_gdf)

cafes_gdf['n_offices_500m'] = count_within_radius(cafes_gdf, office_gdf, 500)
cafes_gdf['n_hospitals_500m'] = count_within_radius(cafes_gdf, medical_gdf, 500)
cafes_gdf['n_schools_500m'] = count_within_radius(cafes_gdf, education_gdf, 500)
cafes_gdf['n_retail_500m'] = count_within_radius(cafes_gdf, retail_gdf, 500)
cafes_gdf['n_residential_500m'] = count_within_radius(cafes_gdf, residential_gdf, 500)
cafes_gdf['n_bus_500m'] = count_within_radius(cafes_gdf, bus_gdf, 500)
cafes_gdf['n_tram_500m'] = count_within_radius(cafes_gdf, tram_gdf, 500)
cafes_gdf['n_metro_500m'] = count_within_radius(cafes_gdf, metro_gdf, 500)
cafes_gdf['n_leisure_500m'] = count_within_radius(cafes_gdf, leisure_gdf, 500)

cafes_gdf['n_offices_300m'] = count_within_radius(cafes_gdf, office_gdf, 300)
cafes_gdf['n_hospitals_300m'] = count_within_radius(cafes_gdf, medical_gdf, 300)
cafes_gdf['n_schools_300m'] = count_within_radius(cafes_gdf, education_gdf, 300)
cafes_gdf['n_retail_300m'] = count_within_radius(cafes_gdf, retail_gdf, 300)
cafes_gdf['n_residential_300m'] = count_within_radius(cafes_gdf, residential_gdf, 300)
cafes_gdf['n_bus_300m'] = count_within_radius(cafes_gdf, bus_gdf, 300)
cafes_gdf['n_tram_300m'] = count_within_radius(cafes_gdf, tram_gdf, 300)
cafes_gdf['n_metro_300m'] = count_within_radius(cafes_gdf, metro_gdf, 300)
cafes_gdf['n_leisure_300m'] = count_within_radius(cafes_gdf, leisure_gdf, 300)

In [6]:
cafes_gdf

,name,type,osm_type,address,cuisine,rating,website,coordinates,lon,lat,...,n_leisure_500m,n_offices_300m,n_hospitals_300m,n_schools_300m,n_retail_300m,n_residential_300m,n_bus_300m,n_tram_300m,n_metro_300m,n_leisure_300m
0,Com,cafe,node,,vietnamese,,,"{'longitude': 37.644618, 'latitude': 55.76828}",37.644618,55.768280,...,6.0,16.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,6.0
1,Шоколадница,cafe,node,,coffee_shop,,,"{'longitude': 37.631644, 'latitude': 55.757364}",37.631644,55.757364,...,32.0,30.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0,16.0
2,Кофе Хауз,cafe,node,,coffee_shop,,,"{'longitude': 37.658665, 'latitude': 55.757643}",37.658665,55.757643,...,15.0,4.0,0.0,0.0,2.0,6.0,3.0,0.0,3.0,14.0
3,Алтаргана,cafe,node,,buryat,,,"{'longitude': 37.652294, 'latitude': 55.767346}",37.652294,55.767346,...,4.0,18.0,2.0,0.0,0.0,3.0,2.0,0.0,0.0,3.0
4,Буржуй,cafe,node,Нижняя Красносельская улица,russian,,www.burjui-club.ru,"{'longitude': 37.67189, 'latitude': 55.774863}",37.671890,55.774863,...,8.0,14.0,0.0,0.0,0.0,7.0,2.0,0.0,0.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,Fruits & Veges,cafe,way,Нижняя Сыромятническая улица,falafel;smoothie;salad;coffee_shop;diet;healthy,,,"{'longitude': 37.66847, 'latitude': 55.752679}",37.668470,55.752679,...,4.0,16.0,0.0,1.0,0.0,2.0,0.0,1.0,0.0,2.0
253,Кооператив «Чёрный»,cafe,way,Лялин переулок,coffee_shop,,,"{'longitude': 37.651832, 'latitude': 55.76002}",37.651832,55.760020,...,7.0,13.0,0.0,1.0,0.0,23.0,0.0,0.0,0.0,5.0
254,Столовая СМ-Энерго,cafe,way,,,,,"{'longitude': 37.690859, 'latitude': 55.769485}",37.690859,55.769485,...,0.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0
255,Sito,cafe,way,,,,,"{'longitude': 37.66088, 'latitude': 55.766124}",37.660880,55.766124,...,4.0,15.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,1.0


In [7]:
cafes_features = cafes_gdf[[
    'name', 'lon', 'lat',
    'dist_to_retail', 'dist_to_residential', 'dist_to_medical',
    'dist_to_leisure', 'dist_to_education', 'dist_to_metro',
    'dist_to_bus', 'dist_to_tram', 'dist_to_office',
    'n_offices_500m', 'n_hospitals_500m', 'n_schools_500m',
    'n_retail_500m', 'n_residential_500m', 'n_bus_500m',
    'n_tram_500m', 'n_metro_500m', 'n_leisure_500m', 'n_offices_300m', 'n_hospitals_300m',
    'n_schools_300m', 'n_retail_300m', 'n_residential_300m', 'n_bus_300m',
    'n_tram_300m', 'n_metro_300m', 'n_leisure_300m'
    
]]

In [17]:
cafes_features

,name,lon,lat,dist_to_retail,dist_to_residential,dist_to_medical,dist_to_leisure,dist_to_education,dist_to_metro,dist_to_bus,...,n_leisure_500m,n_offices_300m,n_hospitals_300m,n_schools_300m,n_retail_300m,n_residential_300m,n_bus_300m,n_tram_300m,n_metro_300m,n_leisure_300m
0,Com,37.644618,55.768280,1491.731567,165.239789,413.117269,18.674639,155.689567,2546.012875,477.047256,...,6.0,16.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,6.0
1,Шоколадница,37.631644,55.757364,847.109780,262.177778,158.369483,90.850518,663.578175,400.569282,130.371773,...,32.0,30.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0,16.0
2,Кофе Хауз,37.658665,55.757643,94.183647,213.838757,440.312001,28.620299,672.629301,79.277015,168.443244,...,15.0,4.0,0.0,0.0,2.0,6.0,3.0,0.0,3.0,14.0
3,Алтаргана,37.652294,55.767346,1558.961273,66.587159,93.487078,175.757577,382.294802,1967.345265,208.184726,...,4.0,18.0,2.0,0.0,0.0,3.0,2.0,0.0,0.0,3.0
4,Буржуй,37.671890,55.774863,468.842768,124.798620,729.713400,115.148164,344.731606,1029.324986,91.576236,...,8.0,14.0,0.0,0.0,0.0,7.0,2.0,0.0,0.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,Fruits & Veges,37.668470,55.752679,953.803650,126.116417,407.679044,100.789652,253.888269,1354.201295,516.007181,...,4.0,16.0,0.0,1.0,0.0,2.0,0.0,1.0,0.0,2.0
253,Кооператив «Чёрный»,37.651832,55.760020,697.461762,27.313838,339.417299,84.014340,204.614972,833.311707,330.475231,...,7.0,13.0,0.0,1.0,0.0,23.0,0.0,0.0,0.0,5.0
254,Столовая СМ-Энерго,37.690859,55.769485,1188.068687,238.054115,884.863534,863.603912,186.153898,1346.057222,141.874013,...,0.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0
255,Sito,37.660880,55.766124,855.847656,151.760972,396.472604,258.942527,336.965029,1626.826873,109.884789,...,4.0,15.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,1.0


In [9]:
cafes_features.describe().T

,count,mean,std,min,25%,50%,75%,max
lon,257.0,37.658923,0.018636,37.629507,37.642661,37.655664,37.674006,37.700424
lat,257.0,55.764209,0.007586,55.752032,55.758645,55.762007,55.770217,55.781962
dist_to_retail,257.0,717.013118,469.545071,0.000000,386.813210,605.248564,973.283352,2304.370114
dist_to_residential,257.0,112.393022,93.051993,4.056984,48.195038,88.369606,152.522123,564.587544
dist_to_medical,257.0,267.686006,186.958164,10.189289,119.034424,233.433376,358.576348,976.369849
dist_to_leisure,257.0,118.628769,135.106264,3.058648,33.175569,78.063210,151.965003,863.603912
dist_to_education,257.0,272.367578,153.506572,8.388324,157.670981,239.075015,372.092493,793.205203
dist_to_metro,257.0,1054.097944,519.143086,12.206161,706.403004,1045.100131,1324.541556,2546.012875
dist_to_bus,257.0,334.854741,277.228933,7.662233,142.835180,244.825866,476.668368,1305.829320
dist_to_tram,257.0,765.240226,587.643364,27.564489,328.012850,581.554949,1078.323953,2704.883024


In [10]:
cafes_features.name

0                      Com
1              Шоколадница
2                Кофе Хауз
3                Алтаргана
4                   Буржуй
              ...         
252         Fruits & Veges
253    Кооператив «Чёрный»
254     Столовая СМ-Энерго
255                   Sito
256               Кислород
Name: name, Length: 257, dtype: object

In [11]:
import folium
from folium.plugins import MarkerCluster

center_lat = cafes_gdf.to_crs(4326)['lat'].mean()
center_lon = cafes_gdf.to_crs(4326)['lon'].mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=14)
marker_cluster = MarkerCluster().add_to(m)

for _, row in cafes_gdf.to_crs(4326).iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=(
            f"<b>{row['name']}</b><br>"
            f"до метро: {row['dist_to_metro']:.0f} м<br>"
            f"до офиса: {row['dist_to_office']:.0f} м<br>"
            f"офисов в 500м: {row['n_offices_500m']:.0f}"
        ),
        icon=folium.Icon(color='red', icon='coffee', prefix='fa')
    ).add_to(marker_cluster)

for _, row in metro_gdf.to_crs(4326).iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        color='blue',
        fill=True,
        fill_opacity=0.7,
        popup=row['name']
    ).add_to(m)
folium.LayerControl(collapsed=False).add_to(m)

m.save('cafes_map.html')
m

In [14]:
import folium
from folium.plugins import MarkerCluster

center_lat = cafes_gdf.to_crs(4326)['lat'].mean()
center_lon = cafes_gdf.to_crs(4326)['lon'].mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=14)
cafes_cluster = MarkerCluster(name='Кафе').add_to(m)
for _, row in cafes_gdf.to_crs(4326).iterrows():
    popup_text = f"""
    Офисов в 500 м: {int(row['n_offices_500m'])}<br>
    Школ в 500 м: {int(row['n_schools_500m'])}<br>
    Больниц в 500 м: {int(row['n_hospitals_500m'])}<br>
    Ближайшее метро: {round(row['dist_to_metro']) if row['dist_to_metro'] else '—'} м
    """
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=popup_text,
        icon=folium.Icon(color='red', icon='coffee', prefix='fa')
    ).add_to(cafes_cluster)

medical_layer = folium.FeatureGroup(name='Медицинские учреждения').add_to(m)
for _, row in medical_gdf.to_crs(4326).iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=6,
        color='white',
        fill=True,
        fill_opacity=0.8,
        popup=row['name']
    ).add_to(medical_layer)

metro_layer = folium.FeatureGroup(name='Метро').add_to(m)
for _, row in metro_gdf.to_crs(4326).iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=6,
        color='blue',
        fill=True,
        fill_opacity=0.8,
        popup=row['name']
    ).add_to(metro_layer)

bus_layer = folium.FeatureGroup(name='Автобусы').add_to(m)
for _, row in bus_gdf.to_crs(4326).iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=4,
        popup = row['name'],
        color='green',
        fill=True,
        fill_opacity=0.7,
    ).add_to(bus_layer)

education_layer = folium.FeatureGroup(name='Образовательные учреждения').add_to(m)
for _, row in education_gdf.to_crs(4326).iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=4,
        popup = row['name'],
        color='black',
        fill=True,
        fill_opacity=0.7,
    ).add_to(education_layer)

leisure_layer = folium.FeatureGroup(name='Досуг').add_to(m)
for _, row in leisure_gdf.to_crs(4326).iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=4,
        popup = row['name'],
        color='yellow',
        fill=True,
        fill_opacity=0.7,
    ).add_to(leisure_layer)

office_layer = folium.FeatureGroup(name='Офисы').add_to(m)
for _, row in office_gdf.to_crs(4326).iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=5,
        color='purple',
        fill=True,
        fill_opacity=0.7,
    ).add_to(office_layer)

folium.LayerControl(collapsed=False).add_to(m)

m.save('basmannyy_map.html')
m

In [15]:
cafes_features.select_dtypes(include='number').corr()

,lon,lat,dist_to_retail,dist_to_residential,dist_to_medical,dist_to_leisure,dist_to_education,dist_to_metro,dist_to_bus,dist_to_tram,...,n_leisure_500m,n_offices_300m,n_hospitals_300m,n_schools_300m,n_retail_300m,n_residential_300m,n_bus_300m,n_tram_300m,n_metro_300m,n_leisure_300m
lon,1.000000,0.744375,0.176732,0.232793,0.334115,0.407556,0.108751,-0.133118,-0.406469,0.172933,...,-0.717288,-0.380868,-0.319381,-0.238891,0.116222,-0.255360,0.402536,0.073172,0.044989,-0.625776
lat,0.744375,1.000000,0.237592,0.076597,0.272829,0.290014,0.257044,-0.027315,-0.391520,0.293297,...,-0.546911,-0.061251,-0.192159,-0.281700,0.062900,-0.129302,0.437234,0.080353,-0.037585,-0.494130
dist_to_retail,0.176732,0.237592,1.000000,0.098329,0.104575,0.095695,0.217781,0.272766,0.044698,0.693882,...,-0.258251,0.104479,-0.091613,-0.195121,-0.477782,-0.153614,0.049021,-0.197142,-0.171073,-0.246041
dist_to_residential,0.232793,0.076597,0.098329,1.000000,0.300719,0.013262,0.096330,-0.034734,0.013988,0.058856,...,-0.169900,0.100156,-0.284937,-0.192765,0.044702,-0.645021,-0.060573,-0.041518,0.009772,-0.162143
dist_to_medical,0.334115,0.272829,0.104575,0.300719,1.000000,0.394579,0.141658,0.117177,-0.216546,0.022733,...,-0.350512,-0.157439,-0.629991,-0.213780,-0.063474,-0.362649,0.053715,-0.166992,-0.031880,-0.345612
dist_to_leisure,0.407556,0.290014,0.095695,0.013262,0.394579,1.000000,0.122984,0.130017,-0.124499,0.100905,...,-0.468045,-0.280542,-0.197502,-0.073475,-0.047627,-0.102017,0.033915,-0.040131,-0.070424,-0.485461
dist_to_education,0.108751,0.257044,0.217781,0.096330,0.141658,0.122984,1.000000,-0.110542,-0.093089,0.302956,...,-0.119288,0.137236,-0.011516,-0.654808,0.070962,-0.140047,0.139038,0.076616,0.241366,-0.113876
dist_to_metro,-0.133118,-0.027315,0.272766,-0.034734,0.117177,0.130017,-0.110542,1.000000,0.184497,-0.081640,...,-0.272055,0.112238,-0.200163,0.110941,-0.299202,-0.065264,-0.134276,0.163187,-0.387929,-0.239943
dist_to_bus,-0.406469,-0.391520,0.044698,0.013988,-0.216546,-0.124499,-0.093089,0.184497,1.000000,-0.184256,...,0.110559,0.203899,0.277644,0.239026,0.000042,-0.114212,-0.710342,0.166528,-0.101274,0.161179
dist_to_tram,0.172933,0.293297,0.693882,0.058856,0.022733,0.100905,0.302956,-0.081640,-0.184256,1.000000,...,-0.091791,-0.012143,-0.032583,-0.243136,-0.221679,-0.123457,0.195411,-0.456091,-0.034119,-0.094125
